In [1]:
#Standard libraries
import os
import time
import sys
#Third party libraries
import rasterio
import rasterio.plot
import rasterio.mask
import rasterio.io
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(legacy='1.25')
import geopandas
import pandas
import fiona
from tqdm.notebook import tqdm
from shapely.geometry import Polygon
#Local applications
package_dir = os.path.dirname(os.getcwd())
if package_dir not in sys.path:
    sys.path.insert(0, package_dir)
from hpr_detection_toolkit import utils
from hpr_detection_toolkit.line_detection import LineSegmentDetector
from hpr_detection_toolkit.hpr_detection import HprDitchDetector

# Setting up
First we define the coordinate reference system in which we want to work in. 

In [2]:
target_crs = "EPSG:31370"

Next, we get point our program to the necessary data. Later, we load the data using the `utils` module of this package, which will check if the crs of the data sets and performs a reprojection on the fly if needed.

In this example, we will use the vector data of the biological value map (BWK) and the agricultural usages plots (Lgp or Lbgebrperc), and the raster data of the VITO AI map regarding microrelief.

> **Adjust the filepath for your own local set-up!**

In [3]:
bwk_filename = 'E:/Stage INBO/Data/BWK_2_20231107_GML/GML/BwkHab.gml'  # change this file path!
lgp_filename = 'E:/Stage INBO/Data/Landbouwaangifte 2016-2020/Lbgb2016_2020.gdb'  # change this file path!
VITO_filename = 'E:/Stage INBO/Data/VITO_microrelief/KB20_predicted_logits.tif'  # change this file path!
output_filename = 'DetectionMicrorelief_KB20.gpkg'

# Selecting the grasslands
### Masking the search region in the BWK
Not all grasslands should be inspected. The once that are already categorised as valuable don't need to be processed. To evaluate this, we use the BWK, making a filter out of it to select the right grasslands to perform the analyses on. 

1. We should only analyse grasslands that are not a habitat (HAB1 contains `gh` or `rbb`)
2. Grasslands that are already categorised as `hpr`of `hpr+` in EENH1, don't need to be analysed

To easily work with the search and nosearch region, we unify all the polygons into one geometry and store it as a geopandas.DataFrame. This step may already been preprocessed in the past. Therefore we check if the file already exist, and if not we calculate it and store it for future analyses.

In [4]:
preprocessing_filename = 'data/DetectionMicrorelief_preprocessing.gpkg'
preprocessed_layers = {}

layers_to_load = ['search_region', 'nosearch_region']
if os.path.isfile(preprocessing_filename):
    print(f'Preprocessed file detected.')
    print(f'Checking layers in {preprocessing_filename}')
    for layer_name in layers_to_load:
        if layer_name in fiona.listlayers(preprocessing_filename):
            preprocessed_layers[layer_name] = geopandas.read_file(preprocessing_filename, layer=layer_name)
            print(f'Layer {layer_name} found in {preprocessing_filename} and loaded in')

for layer_name in layers_to_load:
    if layer_name not in preprocessed_layers.keys():
        print(f'Layer {layer_name} not found in {preprocessing_filename}')
        if layer_name == 'nosearch_region':
            nosearch_region = bwk[~mask].geometry.union_all()
            preprocessed_layer['nosearch_region'] = geopandas.GeoDataFrame(geometry=[nosearch_region], crs=target_crs)
            preprocessed_layer['nosearch_region'].to_file(preprocessing_filename, driver='GPKG', mode='a', layer='nosearch_region')
            print(f'Layer {layer_name} calculated and saved to {preprocessing_filename}')
        if layer_name == 'search_region':
            search_region = bwk[mask].geometry.union_all()
            preprocessed_layer['search_region'] = geopandas.GeoDataFrame(geometry=[search_region], crs=target_crs)
            preprocessed_layer['search_region'].to_file(preprocessing_filename, driver='GPKG', mode='a', layer='search_region')
            print(f'Layer {layer_name} calculated and saved to {preprocessing_filename}')

Preprocessed file detected.
Checking layers in data/DetectionMicrorelief_preprocessing.gpkg
Layer search_region found in data/DetectionMicrorelief_preprocessing.gpkg and loaded in
Layer nosearch_region found in data/DetectionMicrorelief_preprocessing.gpkg and loaded in


### Loading the permanent grasslands
The plots that are grassland can be obtained from the agriculture usage. We look only at permanent grassland, meaning the landplot has always been categorised as grassland during a certain time period (here 2016 to 2023)

Note: to speed up the calculations, we limit this example to KB20 of the VITO raster map. Therefore, we only use the agricultural land-usage plots located within the boundaries of this raster map.

In [5]:
lgp = utils.open_vector_data(lgp_filename, layer='Lbgebrperc2016_2023', target_crs=target_crs)

Vector data in E:/Stage INBO/Data/Landbouwaangifte 2016-2020/Lbgb2016_2020.gdb in EPSG:31370, no reprojection.


In [6]:
VITO_map = utils.open_raster_data(VITO_filename, target_crs=target_crs)

Raster data in E:/Stage INBO/Data/VITO_microrelief/KB20_predicted_logits.tif in EPSG:31370, no reprojection.


In [7]:
left, bottom, right, top = VITO_map.bounds
KB20_boarder = Polygon([(left, bottom), (left, top), (right, top), (right, bottom)])
lgp = lgp[lgp.within(KB20_boarder)]

In [8]:
mask_grasslands = lgp['lgp_7j_BWK'] == 'Permanent grasland - hp'
grasslands = lgp[mask_grasslands]
print(f'KB20 contains {len(lgp)} landplots, of which {len(grasslands)} ({len(grasslands)/len(lgp)*100:.0f}%) are permanent grasslands')

KB20 contains 44999 landplots, of which 10882 (24%) are permanent grasslands


### Selecting permanent grasslands to analyse.
Next, we check which grassland are already categorised in the BWK as valuable (hpr or habitat, see mask BKW), as these do not need to be processed. We check if this selection was already performed and saved to save time 

In [9]:
grasslands_to_inspect = None

if os.path.isfile(output_filename):
    print(f"Output file '{output_filename}' detected.")
    print(f"Checking if file contains selection of grasslands to inspect.")
    if 'grassland_selection' in fiona.listlayers(output_filename):
        grasslands_to_inspect = geopandas.read_file(output_filename, layer='grassland_selection')
        print(f'Grassland selection found in and loaded in')

if grasslands_to_inspect is None:
    print(f'Evaluating which grasslands to process')
    mask_grasslands = np.zeros(len(grasslands), dtype=bool)
    for i in tqdm(range(len(grasslands)), total=len(grasslands), desc="Selecting grasslands to analyse"):
        intersection = grasslands.iloc[i]['geometry'].intersection(search_region.loc[0,'geometry'])
        overlap_fraction = intersection.area / grasslands.iloc[i]['geometry'].area
        if overlap_fraction > .75:
            mask_grasslands[i] = True
    grasslands_to_inspect = grasslands[mask_grasslands]
    grasslands_to_inspect.to_file(output_filename, driver='GPKG', mode='a', layer='grassland_selection')
    print(f"Grasslands selected and stored to layer 'grassland_selection' in {output_filename}.")
    
print(f'KB20 contains {len(grasslands)} permanent grasslands, of which {len(grasslands_to_inspect)}' + 
      f' ({len(grasslands_to_inspect)/len(grasslands)*100:.0f}%) need to be analysed.')

Output file 'DetectionMicrorelief_KB20.gpkg' detected.
Checking if file contains selection of grasslands to inspect.
Grassland selection found in and loaded in
KB20 contains 10882 permanent grasslands, of which 7816 (72%) need to be analysed.


# Analysing the selected grasslands
The categorisation of the grassland is based on the presence of ditches in the landplot. When a buffer zone around the ditches of 15 m covers 70% of the landplot's area, the grassland can be categorised as HPR. Therefore, we will try to detect the ditches and calculate the buffer fraction, which we store as a new attribute for each landplot. 

First, we add a new attribute to the table for each selected grasslands to hold the area fraction of the buffer zone around the ditches, and we set its value to nan.

In [10]:
grasslands_to_inspect['ditch_buffer_fraction'] = np.nan

To performs the necessary processing step to calculate areal fraction of the buffer zone, the `HprDitchDetector` class from this package can be used

To do so, the `HprDitchDetector` uses some default configuration, which are stored in a yaml file. Nevertheless, these default configurations can be overwritten during initialisation of a new object by including a user-defined configuration (dictionary format).

To detect the ditches, the `HprDitchDetector` relies on a `LineSegmentDetector` object. This also uses default configuration saved in the samen yaml file, which can be overwritten similarly.

In [11]:
lsd = LineSegmentDetector()
user_config = {'buffer_zone': {
                    'distance': 15.},
               'filter_background': {
                    'threshold_value': .5*255}
              }
hpr_detector = HprDitchDetector(VITO_map, lsd, config=user_config)

To calculate the buffer_fraction, we iterate through all the selected landplots and perform the processing. For debugging purposes we will also keep the geometry of the detected ditches and buffer zones. Once processed we save everything to our output geopackage. If you did this already, you could skip the following code block and read in the data directly (see second code block bellow).

In [12]:
ditches = [None] *len(grasslands_to_inspect)
buffer_zones = [None] *len(grasslands_to_inspect)

for i in tqdm(range(len(grasslands_to_inspect)), desc="Processeing selected grasslands"):
    index = grasslands_to_inspect.index[i]
    landplot = grasslands_to_inspect.iloc[i:i+1]
    hpr_detector.process(landplot)
    ditches[i] = hpr_detector.get_ditches(multilinestring=True).geometry.loc[0]
    buffer_zones[i] = hpr_detector.get_buffer_zone().geometry
    if buffer_zones[i] is not None: buffer_zones[i] = buffer_zones[i].loc[0]
    grasslands_to_inspect.loc[index,'ditch_buffer_fraction'] = hpr_detector.get_hpr_fraction()

ditches = geopandas.GeoDataFrame(geometry=ditches, crs=target_crs)
buffer_zones = geopandas.GeoDataFrame(geometry=buffer_zones, crs=target_crs)

grasslands_to_inspect.to_file(output_filename, driver='GPKG', layer='grasslands_processed')
ditches.to_file(output_filename, driver='GPKG', layer='ditches')
buffer_zones.to_file(output_filename, driver='GPKG', layer='buffer_zones')

Processeing selected grasslands:   0%|          | 0/7816 [00:00<?, ?it/s]

It is possible that you have already processed these grasslands earlier. In that case you can just open the saved files

In [13]:
grasslands_to_inspect = utils.open_vector_data(output_filename, layer='grasslands_processed')
ditches = utils.open_vector_data(output_filename, layer='ditches')
buffer_zones = utils.open_vector_data(output_filename, layer='buffer_zones')

Vector data in DetectionMicrorelief_KB20.gpkg in EPSG:31370, no reprojection.
Vector data in DetectionMicrorelief_KB20.gpkg in EPSG:31370, no reprojection.
Vector data in DetectionMicrorelief_KB20.gpkg in EPSG:31370, no reprojection.


In [14]:
print(f'Of the {len(grasslands_to_inspect)} inspected grasslands in KB20, ' + 
      f'{len(grasslands_to_inspect[grasslands_to_inspect.ditch_buffer_fraction > .65])} ' + 
      f'of them are hpr candidates based on ditch detection during this processing')

Of the 7816 inspected grasslands in KB20, 89 of them are hpr candidates based on ditch detection during this processing


Lastly, to visually inspect the grasslands easily, you can make a combined plot of the VITO_map and the detected ditches for landplots with a certain buffer fraction.

In [17]:
plot_directory = os.path.join(os.getcwd(),'plot')
if not os.path.isdir(plot_directory): os.makedirs(plot_directory)
for i in tqdm(range(len(grasslands_to_inspect)), desc="Plotting newly detected hpr grasslands"):
    grassland = grasslands_to_inspect.iloc[i]
    
    if grassland.ditch_buffer_fraction > .65:          
        clipped_image = utils.clip_raster(VITO_map, grassland.geometry.geoms)
    
        fig, ax = plt.subplots()
        # Display the raster image
        rasterio.plot.show(clipped_image, ax=ax, cmap='gray_r')
        # Plot the detected lines
        grasslands_to_inspect[i:i+1].plot(ax=ax, edgecolor='green', lw=1, label='Perceel', facecolor='none')
        buffer_zones.iloc[i:i+1].plot(ax=ax, color='orange', alpha=0.5, label='Buffer')
        ditches.iloc[i:i+1].plot(ax=ax, color='purple', linewidth=2, label='Grachtjes')
        
        fig.suptitle("Detected Lines on Georeferenced Image")
        ax.set_title(f"Ditches detected with a buffer zone fraction of {grassland.ditch_buffer_fraction}")
        ax.set_xlabel("Longitude")
        ax.set_ylabel("Latitude")
    
        fig.savefig(f'plot/KB20_grassland-{grassland.OBJECTID}.png')
        plt.close(fig)

Plotting newly detected hpr grasslands:   0%|          | 0/7816 [00:00<?, ?it/s]